In [3]:
from glob import glob
import polars as pl
import polars.selectors as cs
import gc

In [4]:
%run data_read_common.ipynb

print('---------- data_read')

---------- data_read


In [5]:
train_data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
    ]
}

In [6]:
submit_data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
    ]
}

In [7]:
df_train = feature_eng(**train_data_store)
print("train data shape:\t", df_train.shape)

df_submit = feature_eng(**submit_data_store)
print("submit data shape:\t", df_submit.shape)

train data shape:	 (1526659, 376)
submit data shape:	 (10, 375)


In [8]:
try:
    del train_data_store
except:
    pass;

try:
    del test_data_store
except:
    pass;

gc.collect();

In [9]:
df_train = df_train.pipe(Pipeline.filter_cols)
df_submit = df_submit.select([col for col in df_train.columns if col != "target"])

print("train data shape:\t", df_train.shape)
print("submit data shape:\t", df_submit.shape)

train data shape:	 (1526659, 267)
submit data shape:	 (10, 266)


In [10]:
df_train, cat_cols = to_pandas(df_train)
df_submit, cat_cols = to_pandas(df_submit, cat_cols)

In [11]:
X_train = df_train.drop(columns=["target", "case_id","WEEK_NUM"])
y_train = df_train["target"]
weeks = df_train["WEEK_NUM"]

X_submit = df_submit.drop(columns="WEEK_NUM")
X_submit = X_submit.set_index("case_id")

In [12]:
cat_cols = get_cat_columns(X_train);
num_cols = [col for col in X_train.columns if col not in cat_cols]